# Introduction
This notebook demonstrates the integration capabilities of roboquant with OANDA. OANDA is a broker with a focus on FOREX, but also offers CFD and several commodities. 

There are currently three types of integration available:

1. Back testing with historic data from OANDA 
2. Live testing with live data from OANDA 
3. Paper trading on OANDA (WIP)


By default the required OANDA classes are not imported, so we need to do that first. This will load the Feed and Broker classes for integrating with OANDA.

In [ ]:
%use roboquant
import org.roboquant.oanda.*
Welcome()

In [ ]:
// By default USD are displayed with 2 decimals, but for FOREX trading we want more precision
Currency.USD.defaultFractionDigits = 4

# Back Testing
The first example shows how to use historic data from OANDA in a back test. To use Alpaca you'll need to authenticate with a Key and Secret. There are several ways to pass them to the feed, either directly or through system environment or a property file:


<div class="alert alert-block alert-info">
Make sure you use the demo account since roboquant is still very much in Beta
</div>

In [ ]:
val feed = OANDAHistoricFeed()
val timeFrame = TimeFrame.parse("2021-08-10", "2021-08-11")
feed.retrieveCandles("EUR_USD", "GBP_USD", timeFrame = timeFrame)
feed.assets.summary()

In [ ]:
PriceBarChart(feed, feed.assets.first())

Although trading Forex is just like any another asset class, there are some aspects that are different from assets classes like stocks:

- Being short is as common as being long 
- The spread (for common currency pairs) is lower than most stocks

In [ ]:
fun getRoboquant(strategy: Strategy, vararg metrics: Metric) : Roboquant<MemoryLogger> {
    
    // We allow shorting
    val policy = DefaultPolicy(shorting = true)

    // We use a lower cost model, since the default of 10 BIPS is too much for Forex 
    // We select 2.0 BIPS (OANDA typically is around 1.5 which high peaks of 10.0)
    val costModel = DefaultCostModel(2.0)
    val broker = SimBroker(costModel = costModel)

    return Roboquant(strategy = strategy, AccountSummary(), *metrics, policy = policy, broker = broker)
}

We'll define strategy of which the buy and sell signals are based on candlestick patterns

In [ ]:
val strategy = TAStrategy(15)

// We want to generate a BUY signal if we detect either one of the following wto candlestick patternsL: Morningstar or 3WhiteSoldiers
strategy.buy { ta.cdl3WhiteSoldiers(it) }

// We want to generate a SELL signal if we detect the candlestick pattern Two Crows 
strategy.sell { ta.cdl3BlackCrows(it) }

val roboquant = getRoboquant(strategy)

In [ ]:
roboquant.run(feed)

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
account.portfolio.summary()

In [ ]:
val metric = roboquant.logger.getMetric("account.value")
MetricChart(metric)

In [ ]:
feed.assets.forEach { 
    PriceBarChart(feed, it, account.trades).render()
}

## Results

In [ ]:
val metric = roboquant.logger.getMetric("account.value")
MetricChart(metric)

In [ ]:
feed.assets.forEach { 
    PriceBarChart(feed, it, account.trades).render()
}

# Live Testing
Now we create an instance of the OANDALiveFeed and subscribe to three forex pairs. In order to keep it simple, in this example we subscribe to pairs denoted in USD so we can trade without having to define a currency converter. 
If you run this cell will FOREX exchanges are closed, there will no orders and trades of course.

In [ ]:
val feed = OANDALiveFeed()
feed.subscribeOrderBook("EUR_USD", "GBP_USD", "AUD_USD")

In [ ]:
val strategy = EMACrossover()
val metric = PriceMetric(*feed.assets.toTypedArray())
val roboquant = getRoboquant(strategy, metric)

We have all the components assembled that we need to start the test. All that remains, is to start the run and evaluate the strategy against the feed. We'll run it for 15 minutes, but you can change this. 

In [ ]:
val timeFrame = TimeFrame.nextMinutes(60)
roboquant.run(feed, timeFrame)

## Results
The run has completed, lets see a few result. Of course if the run was outside trading hours without any price action, there will not be much to see.

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
account.portfolio.summary()

In [ ]:
PriceChart(metric, metric.assets.first(), account.trades)

In [ ]:
val logger = roboquant.logger
logger.summary().print()

In [ ]:
logger.getMetricNames().summary()

## Charts
Also lets plot two charts. See also the visualization notebook for examples of how to use charts to display results

In [ ]:
val accountValue = logger.getMetric("account.value")
MetricChart(accountValue, useTime = false)

In [ ]:
TradeChartByAsset(account.trades)

# Paper Trading
And just as it was the case in the previous section, most live feeds will only generate data during trading hours. So if you run these code cells outside trading hours, you won't see signals and orders being generated.

<div class="alert alert-block alert-warning">
The integration with the trading API is still very much work in progress, so only use this with a paper trading account
</div>

In [ ]:
val currencyConverter = FixedExchangeRates(Currency.EUR, 
                                          Currency.USD to 0.9, 
                                          Currency.GBP to 1.2)
val broker = OANDABroker(currencyConverter = currencyConverter)
val account = broker.account

In [ ]:
account.summary()

In [ ]:
account.portfolio.summary()

In [ ]:
val strategy = EMACrossover(3, 5)
val policy = DefaultPolicy(shorting = true)
val roboquant = Roboquant(strategy, AccountSummary(), ProgressMetric(), policy = policy, broker = broker)

The setup of the feed is exactly the same as before. There is no difference in the feed if you use it for live testing with the builtin Simulated Broker or paper- and live-trading. Alos this time we'll run it for 30 minutes.

In [ ]:
val feed = OANDALiveFeed()
val symbols = account.portfolio.assets.map { it.symbol }.toTypedArray()
feed.subscribeOrderBook(*symbols)

In [ ]:
val timeFrame = TimeFrame.nextMinutes(5)
roboquant.run(feed, timeFrame)

## Results

In [ ]:
account.summary()